In [117]:
import os, shutil, re
from helpers import *
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from scipy.io import wavfile
import librosa
import librosa.display
import datetime

%matplotlib inline

In [118]:
############
# Input VARS
TODAY = False
VALID_LABELS = ["lights-on", "turn-off"]
PERSON = "taylor"
TRYS = 1

############
# const VARS

DATE = datetime.datetime.today().strftime('%m_%d')
if TODAY == True:
    FNAME = "{}_{}_0{}/".format(PERSON,DATE,TRYS)
else:
    FNAME = "osc_test/"
ROOT = os.getcwd() + "/audio/" + FNAME
IMG_ROOT = os.getcwd() + "/images_scaled/" + FNAME + "/"
pattern = "[0-9]{2}_[0-9]{2}"
IMG_EXT = ".png"
VERBOSITY = 1000

# """
# Expected directory structure:
# [INSIDE ROOT DIRECTORY]
# ---- [category] voice, no_voice
# -------- [date] 07_02, 07_09, ...
# ------------ [label] down, go, ...
# ---------------- [channel] ch1, ch2, ...
# -------------------- [wave files] *.wav
# """
print(ROOT)

/Users/kyy/cerebro_train/audio/osc_test/


In [119]:
def preprocess(samples, sample_rate, multiplier=1):
    sr = sample_rate * multiplier
    padded = np.zeros(sr)
    samples = samples[:sr]
    padded[:samples.shape[0]] = samples
    return padded

def make_dir(path):
    if not os.path.isdir(path):
        os.mkdir(path)

In [120]:
def move_data_window(samples, sample_rate, imgpath):
    dataPts = len(samples)
    window = 12000 #round(dataPts * .8)
    difference = 4000 #dataPts - window
    chunk = int(difference / 5) 

    for i in range(5):
        x = i * chunk
        changed = samples[x:window+x]
        changed = preprocess(changed, sample_rate, 1)
        S = librosa.feature.melspectrogram(changed, sr=sample_rate, n_mels=128, fmax=512)
        log_S = librosa.power_to_db(S, ref=np.max)
        fig = plt.figure(figsize=(1.28, 1.28), dpi=100, frameon=False)
        ax = plt.Axes(fig, [0., 0., 1., 1.])
        ax.set_axis_off()
        fig.add_axes(ax)
        plt.axis('off')
        librosa.display.specshow(log_S)                          
        plt.savefig(imgpath + "{}".format(i) + IMG_EXT)
        plt.close()

In [121]:
# def process(input_dir, output_dir, overwrite=False):
#     items = 0
#     created = 0
#     found = 0
#     date_mult = {"08_08":2, "08_11":2, "08_14":2}
#     plt.ioff()
#     for date in [x for x in os.listdir(input_dir) if re.match(pattern, x)]:
#         multiplier = 1
#         if date in date_mult:
#             multiplier = date_mult[date]
#         date_path = os.path.join(input_dir, date)
#         o_date_path = os.path.join(output_dir, date)
#         make_dir(o_date_path)
#         for label in [d for d in os.listdir(date_path) if d in VALID_LABELS]:
#             label_path = os.path.join(date_path, label)
#             o_label_path = os.path.join(o_date_path, label)
#             make_dir(o_label_path)
#             print("\tProcessing: {}".format(label_path))
#             print("\tTime: {}".format(curr_time()))
#             for channel in [d for d in os.listdir(label_path) if d.startswith("ch")]:
#                 voice = False
#                 ch = int(channel[2:])
#                 if ch == 4 or ch >= 9:
#                     voice = True
#                 channel_path = os.path.join(label_path, channel)
#                 o_channel_path = os.path.join(o_label_path, channel)
#                 make_dir(o_channel_path)
#                 channel_num = channel[-1]
#                 for file in [f for f in os.listdir(channel_path) if f.endswith(".wav")]:
#                     items += 1
#                     wavpath = os.path.join(channel_path, file)
#                     imgpath = os.path.join(o_channel_path, file[:-4] + IMG_EXT)
#                     if overwrite or not os.path.isfile(imgpath):
#                         created += 1
#                         if items % VERBOSITY == 0:
#                             print("\t\tCreated {}th image".format(items))
#                         sample_rate, samples = wavfile.read(wavpath)
#                         samples = preprocess(samples, sample_rate, multiplier)
#                         # freqs, times, spectrogram = signal.spectrogram(samples, sample_rate)
#                         if voice:
#                             S = librosa.feature.melspectrogram(samples, sr=sample_rate, n_mels=128)
#                         else:
#                             S = librosa.feature.melspectrogram(samples, sr=sample_rate, n_mels=128, fmax=512)
#                         log_S = librosa.power_to_db(S, ref=np.max)
#                         fig = plt.figure(figsize=(1.28, 1.28), dpi=100, frameon=False)
#                         ax = plt.Axes(fig, [0., 0., 1., 1.])
#                         ax.set_axis_off()
#                         fig.add_axes(ax)
#                         plt.axis('off')
#                         librosa.display.specshow(log_S)                          
#                         plt.savefig(imgpath)
#                         plt.close()
#                     else:
#                         found += 1
#                         if items % VERBOSITY == 0:
#                             print("\t\tFound {}th image".format(items))
#     print("\tFound:\t\t{}\n\tCreated:\t{}".format(found, created))
#     plt.ion()

In [122]:
def process(input_dir, output_dir, overwrite=False):
    items = 0
    created = 0
    found = 0
    date_mult = {"08_08":2, "08_11":2, "08_14":2}
    plt.ioff()
    for date in [x for x in os.listdir(input_dir) if re.match(pattern, x)]:
        multiplier = 1
        if date in date_mult:
            multiplier = date_mult[date]
        date_path = os.path.join(input_dir, date)
        o_date_path = os.path.join(output_dir, date)
        make_dir(o_date_path)
        for label in [d for d in os.listdir(date_path) if d in VALID_LABELS]:
            label_path = os.path.join(date_path, label)
            o_label_path = os.path.join(o_date_path, label)
            make_dir(o_label_path)
            print("\tProcessing: {}".format(label_path))
            print("\tTime: {}".format(curr_time()))
            for channel in [d for d in os.listdir(label_path) if d.startswith("ch")]:
                voice = False
                ch = int(channel[2:])
                if ch == 4 or ch >= 9:
                    voice = True
                channel_path = os.path.join(label_path, channel)
                o_channel_path = os.path.join(o_label_path, channel)
                make_dir(o_channel_path)
                channel_num = channel[-1]
                for file in [f for f in os.listdir(channel_path) if f.endswith(".wav")]:
                    items += 1
                    wavpath = os.path.join(channel_path, file)
                    imgpath = os.path.join(o_channel_path, file[:-4])
                    if overwrite or not os.path.isfile(imgpath):
                        created += 1
                        if items % VERBOSITY == 0:
                            print("\t\tCreated {}th image".format(items))
                        sample_rate, samples = wavfile.read(wavpath)
                        move_data_window(samples, sample_rate, imgpath)
                        # freqs, times, spectrogram = signal.spectrogram(samples, sample_rate)
                    else:
                        found += 1
                        if items % VERBOSITY == 0:
                            print("\t\tFound {}th image".format(items))
    print("\tFound:\t\t{}\n\tCreated:\t{}".format(found, created))
    plt.ion()

In [123]:
##############################################
# converts splitted wavfiles into spectrograms

if not os.path.isdir(IMG_ROOT):
    os.mkdir(IMG_ROOT)

dir_pairs = {
    ROOT+"voice":IMG_ROOT+"voice",
    ROOT+"no_voice":IMG_ROOT+"no_voice"
}

In [124]:
for input_dir in dir_pairs:
    output_dir = dir_pairs[input_dir]
    if not os.path.isdir(output_dir):
        os.mkdir(output_dir)
    timer(process, input_dir, output_dir)

Start: 2018-12-12 14:27:34.040182
	Found:		0
	Created:	0
End: 2018-12-12 14:27:34.041959
Finished in 00:00:00:00.00164
Start: 2018-12-12 14:27:34.042808
	Processing: /Users/kyy/cerebro_train/audio/osc_test/no_voice/12_11/lights-on
	Time: 2018-12-12 14:27:34.045339
		Created 1000th image
	Processing: /Users/kyy/cerebro_train/audio/osc_test/no_voice/12_11/turn-off
	Time: 2018-12-12 14:30:55.780194
		Created 2000th image
		Created 3000th image
	Processing: /Users/kyy/cerebro_train/audio/osc_test/no_voice/12_13/lights-on
	Time: 2018-12-12 14:34:12.344355
		Created 4000th image
		Created 5000th image
	Processing: /Users/kyy/cerebro_train/audio/osc_test/no_voice/12_13/turn-off
	Time: 2018-12-12 14:37:30.261368
		Created 6000th image
		Created 7000th image
	Processing: /Users/kyy/cerebro_train/audio/osc_test/no_voice/12_14/lights-on
	Time: 2018-12-12 14:40:53.247679
		Created 8000th image
	Processing: /Users/kyy/cerebro_train/audio/osc_test/no_voice/12_14/turn-off
	Time: 2018-12-12 14:44:10.4